# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install yfinance numpy pandas scipy XlsxWriter ipython

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [4]:
my_columns = ['Ticker', 'Stock Price', '1 Year Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        '1 Year Return': [yr1Return],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  1 Year Return Number of Shares to Buy
0        A     109.8100      -0.155167                     N/A
1      AAL      12.8150       0.000000                     N/A
2      AAP      51.7686      -0.682815                     N/A
3     AAPL     173.0400       0.194114                     N/A
4     ABBV     148.1557       0.052748                     N/A
..     ...          ...            ...                     ...
498    YUM     122.6650       0.121776                     N/A
499    ZBH     108.8300      -0.008228                     N/A
500   ZBRA     221.6300      -0.176244                     N/A
501   ZION      33.5600      -0.364053                     N/A
502    ZTS     170.7800       0.120585                     N/A

[503 rows x 4 columns]


In [6]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,109.8100,-0.155167,N/A
1,AAL,12.8150,0.000000,N/A
2,AAP,51.7686,-0.682815,N/A
3,AAPL,173.0400,0.194114,N/A
4,ABBV,148.1557,0.052748,N/A
...,...,...,...,...
498,YUM,122.6650,0.121776,N/A
499,ZBH,108.8300,-0.008228,N/A
500,ZBRA,221.6300,-0.176244,N/A
501,ZION,33.5600,-0.364053,N/A


In [7]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
80,CAT,262.13,NaN,N/A


Filling the `NULL` cells of the `1 Year Return` column with mean value

In [8]:
final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)

In [9]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


In [10]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,109.8100,-0.155167,N/A
1,AAL,12.8150,0.000000,N/A
2,AAP,51.7686,-0.682815,N/A
3,AAPL,173.0400,0.194114,N/A
4,ABBV,148.1557,0.052748,N/A
...,...,...,...,...
498,YUM,122.6650,0.121776,N/A
499,ZBH,108.8300,-0.008228,N/A
500,ZBRA,221.6300,-0.176244,N/A
501,ZION,33.5600,-0.364053,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [11]:
final_dataframe.sort_values('1 Year Return', ascending=False, inplace=True)

In [12]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [13]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,441.5850,2.354227,N/A
1,META,300.3750,1.197311,N/A
2,GE,108.0300,1.105661,N/A
3,FICO,855.5300,1.080252,N/A
4,RCL,89.7650,1.029986,N/A
5,CCL,13.2550,0.875358,N/A
6,BKNG,3010.5400,0.781843,N/A
7,PHM,72.2900,0.753332,N/A
8,ADBE,510.2150,0.737274,N/A
9,AVGO,816.8700,0.718060,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [14]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [16]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,441.5850,2.354227,452
1,META,300.3750,1.197311,665
2,GE,108.0300,1.105661,1851
3,FICO,855.5300,1.080252,233
4,RCL,89.7650,1.029986,2228
5,CCL,13.2550,0.875358,15088
6,BKNG,3010.5400,0.781843,66
7,PHM,72.2900,0.753332,2766
8,ADBE,510.2150,0.737274,391
9,AVGO,816.8700,0.718060,244


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is that low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Here `hqm` stands for `high-quality momentum`.

In [17]:
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    '1 Year Return',
    '1 Year Return Percentile',
    '6 Months Return',
    '6 Months Return Percentile',
    '3 Months Return',
    '3 Months Return Percentile',
    '1 Month Return',
    '1 Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [18]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='6mo')
        mnth6Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth6Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='3mo')
        mnth3Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth3Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='1mo')
        mnth1Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth1Return = np.NaN
    
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        'Number of Shares to Buy': ['N/A'],
        '1 Year Return': [yr1Return],
        '1 Year Return Percentile': ['N/A'],
        '6 Months Return': [mnth6Return],
        '6 Months Return Percentile': ['N/A'],
        '3 Months Return': [mnth3Return],
        '3 Months Return Percentile': ['N/A'],
        '1 Month Return': [mnth1Return],
        '1 Month Return Percentile': ['N/A'],
        'HQM Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    hqm_dataframe = pd.concat([hqm_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(hqm_dataframe['Ticker'])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
498     YUM
499     ZBH
500    ZBRA
501    ZION
502     ZTS
Name: Ticker, Length: 503, dtype: object


In [19]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,109.5100,N/A,-0.155167,N/A,-20.385006,N/A,-8.175068,N/A,-7.579558,N/A,N/A
1,AAL,12.8173,N/A,0.000000,N/A,-7.722826,N/A,-30.036572,N/A,-10.368534,N/A,N/A
2,AAP,51.6900,N/A,-0.682815,N/A,-56.307647,N/A,-27.103296,N/A,-23.490230,N/A,N/A
3,AAPL,172.9499,N/A,0.194114,N/A,5.900766,N/A,-9.489604,N/A,-8.829780,N/A,N/A
4,ABBV,148.0850,N/A,0.052748,N/A,-6.029803,N/A,8.797194,N/A,1.358659,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.7500,N/A,0.121776,N/A,-6.093253,N/A,-9.757733,N/A,-4.608325,N/A,N/A
499,ZBH,108.7750,N/A,-0.008228,N/A,-15.084781,N/A,-23.425674,N/A,-7.468621,N/A,N/A
500,ZBRA,220.5400,N/A,-0.176244,N/A,-26.523406,N/A,-24.691826,N/A,-19.290030,N/A,N/A
501,ZION,33.4900,N/A,-0.364053,N/A,20.410872,N/A,21.168674,N/A,-7.306945,N/A,N/A


In [20]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
80,CAT,262.32,N/A,NaN,N/A,24.127767,N/A,7.674336,N/A,-6.856513,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [21]:
hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)

In [22]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [23]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,109.5100,N/A,-0.155167,N/A,-20.385006,N/A,-8.175068,N/A,-7.579558,N/A,N/A
1,AAL,12.8173,N/A,0.000000,N/A,-7.722826,N/A,-30.036572,N/A,-10.368534,N/A,N/A
2,AAP,51.6900,N/A,-0.682815,N/A,-56.307647,N/A,-27.103296,N/A,-23.490230,N/A,N/A
3,AAPL,172.9499,N/A,0.194114,N/A,5.900766,N/A,-9.489604,N/A,-8.829780,N/A,N/A
4,ABBV,148.0850,N/A,0.052748,N/A,-6.029803,N/A,8.797194,N/A,1.358659,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.7500,N/A,0.121776,N/A,-6.093253,N/A,-9.757733,N/A,-4.608325,N/A,N/A
499,ZBH,108.7750,N/A,-0.008228,N/A,-15.084781,N/A,-23.425674,N/A,-7.468621,N/A,N/A
500,ZBRA,220.5400,N/A,-0.176244,N/A,-26.523406,N/A,-24.691826,N/A,-19.290030,N/A,N/A
501,ZION,33.4900,N/A,-0.364053,N/A,20.410872,N/A,21.168674,N/A,-7.306945,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [24]:
time_periods = [
    '1 Year',
    '6 Months',
    '3 Months',
    '1 Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[change_col],
                                                                           hqm_dataframe.loc[row, change_col]) / 100

In [25]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,109.5100,N/A,-0.155167,0.178926,-20.385006,0.131213,-8.175068,0.485089,-7.579558,0.389662,N/A
1,AAL,12.8173,N/A,0.000000,0.427435,-7.722826,0.369781,-30.036572,0.027833,-10.368534,0.214712,N/A
2,AAP,51.6900,N/A,-0.682815,0.001988,-56.307647,0.003976,-27.103296,0.04175,-23.490230,0.013917,N/A
3,AAPL,172.9499,N/A,0.194114,0.729622,5.900766,0.67992,-9.489604,0.427435,-8.829780,0.296223,N/A
4,ABBV,148.0850,N/A,0.052748,0.522863,-6.029803,0.407555,8.797194,0.902584,1.358659,0.926441,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.7500,N/A,0.121776,0.614314,-6.093253,0.405567,-9.757733,0.413519,-4.608325,0.648111,N/A
499,ZBH,108.7750,N/A,-0.008228,0.411531,-15.084781,0.190855,-23.425674,0.063618,-7.468621,0.399602,N/A
500,ZBRA,220.5400,N/A,-0.176244,0.149105,-26.523406,0.075547,-24.691826,0.057654,-19.290030,0.031809,N/A
501,ZION,33.4900,N/A,-0.364053,0.043738,20.410872,0.914513,21.168674,0.994036,-7.306945,0.425447,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [26]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [27]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,109.5100,N/A,-0.155167,0.178926,-20.385006,0.131213,-8.175068,0.485089,-7.579558,0.389662,0.296223
1,AAL,12.8173,N/A,0.000000,0.427435,-7.722826,0.369781,-30.036572,0.027833,-10.368534,0.214712,0.25994
2,AAP,51.6900,N/A,-0.682815,0.001988,-56.307647,0.003976,-27.103296,0.04175,-23.490230,0.013917,0.015408
3,AAPL,172.9499,N/A,0.194114,0.729622,5.900766,0.67992,-9.489604,0.427435,-8.829780,0.296223,0.5333
4,ABBV,148.0850,N/A,0.052748,0.522863,-6.029803,0.407555,8.797194,0.902584,1.358659,0.926441,0.689861
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.7500,N/A,0.121776,0.614314,-6.093253,0.405567,-9.757733,0.413519,-4.608325,0.648111,0.520378
499,ZBH,108.7750,N/A,-0.008228,0.411531,-15.084781,0.190855,-23.425674,0.063618,-7.468621,0.399602,0.266402
500,ZBRA,220.5400,N/A,-0.176244,0.149105,-26.523406,0.075547,-24.691826,0.057654,-19.290030,0.031809,0.078529
501,ZION,33.4900,N/A,-0.364053,0.043738,20.410872,0.914513,21.168674,0.994036,-7.306945,0.425447,0.594433


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [28]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [29]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [30]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ACGL,81.4200,N/A,0.676250,0.980119,18.705354,0.902584,10.594940,0.930417,7.329288,0.992048,0.951292
1,CHTR,431.6500,N/A,0.336132,0.894632,22.238899,0.936382,17.407861,0.984095,5.540479,0.974155,0.947316
2,CBOE,158.9511,N/A,0.314596,0.866799,17.203290,0.88668,15.123551,0.972167,6.678588,0.986083,0.927932
3,AKAM,105.9100,N/A,0.261724,0.811133,36.816947,0.988072,17.390825,0.982107,1.300814,0.920477,0.925447
4,PCAR,85.6600,N/A,0.449331,0.942346,25.918307,0.95825,4.439398,0.829026,4.898359,0.972167,0.925447
5,META,299.9852,N/A,1.197311,0.998012,41.852656,0.994036,1.909160,0.785288,-0.053308,0.884692,0.915507
6,CEG,107.1750,N/A,0.230413,0.769384,40.973082,0.992048,16.628658,0.980119,1.060820,0.912525,0.913519
7,AFL,76.7600,N/A,0.286270,0.84493,20.523122,0.916501,10.421233,0.928429,3.450142,0.954274,0.911034
8,WDC,46.0500,N/A,0.222072,0.757455,31.346256,0.980119,22.278278,0.996024,0.567806,0.904573,0.909543
9,PKG,152.6300,N/A,0.322723,0.882704,10.681784,0.779324,18.368266,0.99006,5.768447,0.978131,0.907555


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [31]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [32]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

200000.0

In [33]:
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Stock Price'])

hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,ACGL,81.4200,2456,0.676250,0.980119,18.705354,0.902584,10.594940,0.930417,7.329288,0.992048,0.951292
1,CHTR,431.6500,463,0.336132,0.894632,22.238899,0.936382,17.407861,0.984095,5.540479,0.974155,0.947316
2,CBOE,158.9511,1258,0.314596,0.866799,17.203290,0.88668,15.123551,0.972167,6.678588,0.986083,0.927932
3,AKAM,105.9100,1888,0.261724,0.811133,36.816947,0.988072,17.390825,0.982107,1.300814,0.920477,0.925447
4,PCAR,85.6600,2334,0.449331,0.942346,25.918307,0.95825,4.439398,0.829026,4.898359,0.972167,0.925447
5,META,299.9852,666,1.197311,0.998012,41.852656,0.994036,1.909160,0.785288,-0.053308,0.884692,0.915507
6,CEG,107.1750,1866,0.230413,0.769384,40.973082,0.992048,16.628658,0.980119,1.060820,0.912525,0.913519
7,AFL,76.7600,2605,0.286270,0.84493,20.523122,0.916501,10.421233,0.928429,3.450142,0.954274,0.911034
8,WDC,46.0500,4343,0.222072,0.757455,31.346256,0.980119,22.278278,0.996024,0.567806,0.904573,0.909543
9,PKG,152.6300,1310,0.322723,0.882704,10.681784,0.779324,18.368266,0.99006,5.768447,0.978131,0.907555


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [34]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Formatting Our Excel Output

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [37]:
writer.close()